<a href="https://colab.research.google.com/github/ThousandAI/Application-of-AI/blob/main/class02/class02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Autograd

In [1]:
import torch
import torch.nn as nn

In [3]:
# grad = True
tensor = torch.rand(3, requires_grad=True)
print(tensor)
y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)
print(tensor.grad) # 2*tensor + 12

tensor([0.4864, 0.1889, 0.0112], requires_grad=True)
tensor([6.4864, 6.1889, 6.0112], grad_fn=<AddBackward0>)
tensor([128.2195, 116.9068, 110.4040], grad_fn=<AddBackward0>)
tensor(118.5101, grad_fn=<MeanBackward0>)
tensor([12.9728, 12.3778, 12.0224])


In [4]:
# grad = False
tensor = torch.rand(3, requires_grad=False)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)

tensor([0.9621, 0.2801, 0.9149])
tensor([6.9621, 6.2801, 6.9149])
tensor([147.4122, 120.3188, 145.4484])
tensor(137.7265)


RuntimeError: ignored

In [5]:
# not scalar
tensor = torch.tensor([0.37, 0.58, 0.33], requires_grad=True)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z.backward()

tensor([0.3700, 0.5800, 0.3300], requires_grad=True)
tensor([6.3700, 6.5800, 6.3300], grad_fn=<AddBackward0>)
tensor([123.7307, 131.8892, 122.2067], grad_fn=<AddBackward0>)


RuntimeError: ignored

In [6]:
# grad 
tensor = torch.rand(3, requires_grad = True)
print(tensor)
y = tensor.detach()
print(y)

tensor([0.7450, 0.4478, 0.1480], requires_grad=True)
tensor([0.7450, 0.4478, 0.1480])


In [ ]:
# grad 
with torch.no_grad():
  y = tensor + 2
  print(y)

tensor([2.7110, 2.0037, 2.2208])


In [7]:
# accumulated gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])
tensor([12., 12., 12., 12.])
tensor([15., 15., 15., 15.])


In [8]:
# empty gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

  weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


## toy example

In [11]:
# toy example
x = torch.tensor([[1,-1], [2,3], [5,2]], dtype=torch.float32) # 3x2
y = torch.tensor([[1],[0],[1]], dtype=torch.float32)

w1 = torch.rand(2,3, requires_grad=True)
w2 = torch.rand(3,3, requires_grad=True)
w3 = torch.rand(3,2, requires_grad=True)
w4 = torch.rand(2,1, requires_grad=True)
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
bce = nn.BCELoss()

def forward(inputs):
    inputs = torch.matmul(inputs, w1)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w2)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w3)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w4)
    outputs = sigmoid(inputs)
    return outputs

# loss
def loss(y_true, y_pred):
    return bce(y_pred, y_true)


learning_rate = 0.01
epochs = 10

for epoch in range(epochs):
    # forward pass
    y_hat = forward(inputs=x)

    # loss
    bce_loss = loss(y_true=y, y_pred=y_hat)

    # backward loss
    bce_loss.backward()

    # update weights
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w3 -= learning_rate * w3.grad
        w4 -= learning_rate * w4.grad

    # zero gradients
    w1.grad.zero_()
    w2.grad.zero_()
    w3.grad.zero_()
    w4.grad.zero_()

    
    print(f"epoch {epoch + 1}: \nw1 = {w1}\n w2 = {w2}\n w3 = {w3}\n w4 = {w4}, loss = {bce_loss:.8f}")

epoch 1: 
w1 = tensor([[0.1328, 0.5382, 0.5504],
        [0.1804, 0.2279, 0.2337]], requires_grad=True)
 w2 = tensor([[0.2289, 0.5638, 0.1632],
        [0.3929, 0.2375, 0.9889],
        [0.4418, 0.8465, 0.7737]], requires_grad=True)
 w3 = tensor([[0.8405, 0.9692],
        [0.5840, 0.0423],
        [0.6044, 0.0615]], requires_grad=True)
 w4 = tensor([[0.7141],
        [0.8803]], requires_grad=True), loss = 1.96304989
epoch 2: 
w1 = tensor([[0.1284, 0.5317, 0.5425],
        [0.1738, 0.2152, 0.2183]], requires_grad=True)
 w2 = tensor([[0.2250, 0.5626, 0.1619],
        [0.3849, 0.2350, 0.9862],
        [0.4336, 0.8439, 0.7710]], requires_grad=True)
 w3 = tensor([[0.8367, 0.9646],
        [0.5786, 0.0356],
        [0.5970, 0.0524]], requires_grad=True)
 w4 = tensor([[0.6990],
        [0.8743]], requires_grad=True), loss = 1.82733953
epoch 3: 
w1 = tensor([[0.1241, 0.5255, 0.5350],
        [0.1674, 0.2030, 0.2036]], requires_grad=True)
 w2 = tensor([[0.2213, 0.5615, 0.1607],
        [0.3773,

# Dataset & DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class ScoreDataset(Dataset):
  def __init__(self):
    self.x = torch.tensor([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15],[16,17,18]]) # 6x3
    self.y = torch.tensor([[1],[0],[1],[1],[1],[0]]) # 6x1

  def __getitem__(self,index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.x.size(0)

In [ ]:
score_dataset = ScoreDataset()

In [ ]:
print(score_dataset.x)
print(score_dataset.y)

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]])
tensor([[1],
        [0],
        [1],
        [1],
        [1],
        [0]])


In [ ]:
dataLoader = DataLoader(dataset=score_dataset,batch_size=2,shuffle=True)

In [ ]:
for i, (x,y) in enumerate(dataLoader):
  print(f"{i}: x={x}, y={y}")

0: x=tensor([[ 7,  8,  9],
        [10, 11, 12]]), y=tensor([[1],
        [1]])
1: x=tensor([[4, 5, 6],
        [1, 2, 3]]), y=tensor([[0],
        [1]])
2: x=tensor([[13, 14, 15],
        [16, 17, 18]]), y=tensor([[1],
        [0]])
